<a href="https://colab.research.google.com/github/jorgg3/Proyecto-3/blob/main/Ejercicio_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3: Ajuste por Mínimos Cuadrados y Factorización QR
**Problemas Computacionales**\
Alumno: Martínez de la Cruz José Jorge\
Profesor: César Carreón Otañez\
Ayudante:  Jesús Iván Coss Calderón

Para mostrar la diferencia numérica entre el método de las Ecuaciones Normales y la factorización QR se requiere un problema de mínimos cuadrados que sea mal condicionado. Para ello consideremos el ajuste del siguiente polinomio de grado $n-1$,
$$
p_{n-1}(t) =x_n t^{n-1} + x_{n-1} t^{n-2} + \cdots + x_2 t + x_1
$$


Para $m$ datos $(t_i, y_i)$,  $m > n$. Elegimos $t_i = (i − 1)/(m − 1)$,  $i = 1,..., m$. Los valores para $y_i$ serán los dados al evaluar el polinomio con las $t_i$ dadas previamente y tomando $x_j = 1$ para $j = 1, ... , n$, se quiere ver si se pueden recuperar los valores de las $x_j$ con los métodos estudiados.
Primero se genera un perturbación para los valores $y_i$ dado por
$$
y_i = y_i + (2u_i - 1) \cdot \epsilon, \quad i = 1, \dots, m
$$
con $u_i \in [0, 1]$ números aleatorios. Usar $m = 21$, $n = 12$ y $ε = 10^{−10}$

In [27]:
import numpy as np

In [9]:
T = []
for i in range (21):
  ti = (i-1)/20
  T.append(ti)

In [10]:
def polinomio(t):
    resultado = 0
    # Evaluamos el polinomio usando el valor de t
    for i in range(12):
        resultado += t**(12-i-1)

    return resultado

In [21]:
Y = []
for i in range (21):
  yi = polinomio(T[i])
  Y.append(yi)

In [22]:
import random
U = []
for i in range (21):
  ui = random.random()
  U.append(ui)
E = 10**(-10)
for i in range(21):
  Y[i] = Y[i] + (2*U[i]-1)*E

Después de generar la lista de datos $(t_i, y_i)$ se comparan los dos métodos para ajustar el polinomio. Primero, formar el Sistema de Ecuaciones Normales para el problema y resolverlo por la Factorización de Cholesky. Segundo utilizar algún método para factorizar la matriz en una QR y resolver.

In [28]:
matriz = []
for t in T:
    fila = [t**i for i in range(12)]
    matriz.append(fila)
A = np.array(matriz)

Importemos todos nuestros programas para resolver por ecuaciones normales:

In [29]:

def SustDelante(L, b):
    """
    Resuelve un sistema de ecuaciones L * x = b, donde L es una matriz triangular inferior.

    Parámetros:
    L (numpy.ndarray): Matriz triangular inferior.
    b (numpy.ndarray): Vector columna de tamaño n, lado derecho del sistema.

    Retorna:
    x (numpy.ndarray): Vector solución del sistema L * x = b.
    """
    x = np.zeros_like(b)  # Crea un vector x de ceros del mismo tamaño que b
    n = L.shape[0]  # Obtiene el número de renglones de L
    for i in range(n):  # Recorre las filas de L
        sum = 0.0
        for j in range(i):  # Suma los productos de la parte ya resuelta de x
            sum += L[i, j] * x[j]
        # Resuelve para x[i]
        x[i] = (b[i] - sum) / L[i, i]

    return x  # Devuelve la solución

def SustAtras(U, y):
    """
    Resuelve un sistema de ecuaciones U * x = y, donde U es una matriz triangular superior.

    Parámetros:
    U (numpy.ndarray): Matriz triangular superior.
    y (numpy.ndarray): Vector columna de tamaño n, lado derecho del sistema.

    Retorna:
    x (numpy.ndarray): Vector solución del sistema U * x = y.
    """
    x = np.zeros_like(y)  # Crea un vector x de ceros del mismo tamaño que y
    n = U.shape[0]  # Obtiene el número de renglones de U
    x[n-1] = y[n-1] / U[n-1, n-1]  # Resuelve el último valor de x
    for i in range(n-2, -1, -1):  # Recorre las filas desde la penúltima hasta la primera
        sum = 0.0
        for j in range(i+1, n):  # Suma los productos de los valores ya resueltos de x
            sum += U[i, j] * x[j]
        # Resuelve para x[i]
        x[i] = (y[i] - sum) / U[i, i]

    return x  # Devuelve la solución


In [30]:
def Cholesky(A):
    """
    Realiza la descomposición de Cholesky de una matriz simétrica y definida positiva.

    La descomposición de Cholesky de una matriz A se expresa como:
    A = L * L^T
    donde:
        - A es la matriz de entrada (simétrica y definida positiva).
        - L es una matriz triangular inferior (con ceros sobre la diagonal).

    Parámetros:
    A (numpy.ndarray): Matriz cuadrada simétrica y definida positiva de tamaño n x n.

    Retorna:
    L (numpy.ndarray): Matriz triangular inferior de tamaño n x n tal que A = L * L^T.

    Excepciones:
    Si la matriz A no es simétrica o definida positiva, este algoritmo puede fallar
    o devolver resultados incorrectos.

    El algoritmo sigue el siguiente procedimiento:
    1. Inicializa una matriz L de ceros de las mismas dimensiones que A.
    2. Calcula los elementos de la matriz triangular inferior L usando la fórmula de Cholesky.
    3. La diagonal de L es calculada con la raíz cuadrada de los elementos de A.
    4. Los elementos fuera de la diagonal se calculan usando las sumas de productos previamente computadas.

    """
    # Determina el tamaño de la matriz A
    n = len(A)

    # Crea una matriz L de ceros del mismo tamaño que A
    L = np.zeros_like(A)

    # Recorre las filas y columnas de A para calcular los elementos de L
    for i in range(n):
        for j in range(i + 1):
            # Si estamos en la diagonal principal
            if i == j:
                # Calculamos la suma de los cuadrados de los elementos previos de la fila
                sum = 0.0
                for k in range(j):
                    sum += L[j][k] * L[j][k]
                # La raíz cuadrada del valor en A[j][j] menos la suma anterior da el valor de L[j][j]
                L[j][j] = np.sqrt(A[j][j] - sum)
            else:
                # Si estamos en la parte inferior de la matriz (por debajo de la diagonal)
                sum = 0.0
                for k in range(j):
                    sum += L[i][k] * L[j][k]
                # Usamos la fórmula para calcular los valores fuera de la diagonal
                L[i][j] = (A[i][j] - sum) / L[j][j]

    return L

In [31]:
def EcNormal(A, b):
    """
    Resuelve el sistema de ecuaciones lineales A * x = b utilizando el método de los mínimos cuadrados
    con la descomposición de Cholesky de la matriz A^T * A.

    Este método se utiliza cuando el sistema no tiene solución exacta o es sobredeterminado (más ecuaciones que incógnitas).
    En lugar de resolver el sistema directamente, se resuelve el sistema equivalente A^T * A * x = A^T * b,
    que tiene solución en el sentido de los mínimos cuadrados.

    Parámetros:
    A (numpy.ndarray): Matriz de coeficientes del sistema de ecuaciones de tamaño m x n (m >= n).
    b (numpy.ndarray): Vector columna de tamaño m, que es el lado derecho del sistema de ecuaciones.

    Retorna:
    xSol (numpy.ndarray): El vector solución x, que es la solución de mínimos cuadrados al sistema A * x = b.

    Procedimiento:
    1. Calculamos el sistema equivalente A^T * A * x = A^T * b.
    2. Aplicamos la descomposición de Cholesky a la matriz A^T * A para obtener una matriz triangular inferior L.
    3. Resolvemos el sistema triangular inferior L * y = A^T * b mediante sustitución hacia adelante (SustDelante).
    4. Resolvemos el sistema triangular superior L^T * x = y mediante sustitución hacia atrás (SustAtras).
    """

    # Calculamos el sistema equivalente A^T * A * x = A^T * b
    AtA = A.T @ A  # Matriz generalizada (A^T * A)
    Atb = A.T @ b  # Vector del sistema equivalente (A^T * b)

    # Descomposición de Cholesky de A^T * A
    L = Cholesky(AtA)  # L es la matriz triangular inferior tal que A^T * A = L * L^T
    Lt = L.T  # La transpuesta de L, es la matriz triangular superior

    # Resolvemos el sistema L * y = A^T * b mediante sustitución hacia adelante
    ySol = SustDelante(L, Atb)

    # Resolvemos el sistema L^T * x = ySol mediante sustitución hacia atrás
    xSol = SustAtras(Lt, ySol)

    return xSol

Implementamos el programa con la matríz $A$ y las soluciones $Y$

In [48]:
u = EcNormal(A, Y)
print(u)

[1.         0.99999971 1.00000167 1.0000792  0.9987239  1.00846259
 0.96885862 1.06960331 0.90318102 1.08197116 0.96130944 1.00780965]


La solución con el método de ecuaciones normales es:
\begin{bmatrix}
1.00000000 \\
0.99999971 \\
1.00000167 \\
1.00007920 \\
0.99872390 \\
1.00846259 \\
0.96885862 \\
1.06960331 \\
0.90318102 \\
1.08197116 \\
0.96130944 \\
1.00780965
\end{bmatrix}


Realícemos el procedimiento con el método de Gram Smitchd

In [35]:
def QR(A):
    """
    Realiza la descomposición QR de una matriz A utilizando el método de Gram-Schmidt.

    Parámetros:
    A (numpy.ndarray): Matriz de entrada de tamaño (m x n), donde m es el número de filas y n el número de columnas.

    Retorna:
    Q (numpy.ndarray): Matriz ortogonal de tamaño (m x n), tal que Q^T * Q = I.
    R (numpy.ndarray): Matriz triangular superior de tamaño (n x n).

    El resultado satisface la relación A = Q * R, donde:
    - Q es una matriz ortogonal (Q^T * Q = I).
    - R es una matriz triangular superior.
    """

    # Inicialización de las matrices Q y R
    Q = np.empty_like(A)  # Matriz de salida Q, tendrá las mismas dimensiones que A
    R = np.zeros([A.shape[1], A.shape[1]])  # Matriz cuadrada R de tamaño n x n
    vi = np.zeros([A.shape[1]])  # Vector auxiliar vi

    # Iteración sobre las columnas de A
    for i in range(A.shape[1]):
        # Comienza con el vector de la columna i de A
        vi = A[:, i]

        # Proceso de ortogonalización de Gram-Schmidt
        for j in range(i):
            # Calculamos la proyección de vi sobre los vectores Q anteriores
            R[j, i] = np.dot(Q[:, j].T, vi)  # Proyección de vi sobre Q[:, j]
            # Restamos la proyección de vi para mantener la ortogonalidad
            vi = vi - R[j, i] * Q[:, j]

        # Normalizamos el vector vi para obtener un vector unitario
        R[i, i] = np.linalg.norm(vi, 2)  # Norma de vi, la longitud del vector
        Q[:, i] = vi / R[i, i]  # Normalizamos para obtener el i-ésimo vector ortonormal de Q

    # Retorna las matrices Q y R
    return Q, R


In [37]:
def SolGS(A, b):
    """
    Resuelve el sistema de ecuaciones lineales A * x = b utilizando la factorización QR de A.

    Parámetros:
    A (numpy.ndarray): Matriz de coeficientes del sistema de ecuaciones lineales de tamaño (m x n),
                        donde m es el número de ecuaciones y n el número de incógnitas.
    b (numpy.ndarray): Vector de términos independientes del sistema de tamaño (m,).

    Retorna:
    x (numpy.ndarray): Solución del sistema de ecuaciones de tamaño (n,), tal que A * x = b.


    Después de realizar la descomposición QR, se resuelve el sistema Rx = Q^T * b. Este sistema triangular superior se resuelve usando
    la sustitución hacia atrás. La solución se obtiene en dos pasos:

    1. Calcular Q^T * b (esto se almacena en el vector bR).
    2. Resolver el sistema triangular superior Rx = bR usando la sustitución hacia atrás.

    """

    # Paso 1: Obtener la factorización QR de A
    Q, R = QR(A)

    # Paso 2: Calcular el vector bR = Q^T * b
    bR = Q.T @ b

    # Paso 3: Resolver el sistema triangular superior Rx = bR usando sustitución hacia atrás
    x = SustAtras(R, bR)

    return x


In [46]:
v =SolGS(A, Y)
print(v)


[1.         1.00000006 0.9999993  0.99998411 1.00032356 0.99763467
 1.00926428 0.97833326 1.03121236 0.97282337 1.01312568 0.99729923]


En este caso, el vector solución es:
\begin{bmatrix}
1.00000000 \\
1.00000006 \\
0.99999930 \\
0.99998411 \\
1.00032356 \\
0.99763467 \\
1.00926428 \\
0.97833326 \\
1.03121236 \\
0.97282337 \\
1.01312568 \\
0.99729923
\end{bmatrix}



Para responder a las preguntas, calcularemos la norma entre el vector solución

In [47]:
z = np.ones_like(v)

normaGS = np.linalg.norm(v - z)

normaGS


0.049531127858416756

In [49]:
normaEN = np.linalg.norm(u- z)
normaEN

0.15342357214889343

a) ¿Para cuál de los métodos la solución es más sensible a la perturbación generada? Dada la magnitud de las normas, para el sistema resuelto con ecuaciones normales

b) ¿Cuál de los métodos está más proximo a tener la solución exacta dada por $x_i= 1$ Dada la norma, por el de Gram Smitchd

c) ¿La diferencia en las soluciones afecta en el ajuste de puntos $(ti, yi)$ por el polinomio, por qué?. Al obtenerse nuevos coeficientes, entonces el polinomio cambia, lo que indica que dependiendo el método obtendremos un vector solución diferente, es decir, que tendremos un ajuste de puntos diferente, estó está dado por la perturbación que en ambos casos fue distinta.